# DLT24: Impact of Layer-1 Characteristics on Scalability of Layer-2 Semi-Hierarchical Payment Channel Networks

## Initialization

In [ ]:
import sys, os 
from datetime import datetime
import pandas as pd
import shutil
import pathlib

%load_ext autoreload
%autoreload 2

In [ ]:
experiment_root_dir = os.path.abspath(os.path.join(os.getcwd()))
cloth_root_dir = os.path.abspath(os.path.join(os.getcwd(),"../.."))
cloth_root_dir, experiment_root_dir

In [ ]:
from experiments_runner import run_all_simulations
from plasma_network_generator.commands.generate_all import Args as TopologyGeneratorArgs, _execute as topology_generate, DEFAULT_FRACTION_OF_UNBANKED_RETAIL_USERS
from plasma_network_generator.core import select_eurosystem_subset

In [ ]:
seeds = [111, 128, 209, 250, 302, 421, 634, 710, 892, 901]
capacities = [0.001, 0.002, 0.005]

## Create topologies

In [ ]:
topologies_dir = os.path.abspath(os.path.join(cloth_root_dir,f"experiments/2024_DLT/topologies"))
pathlib.Path(topologies_dir).mkdir(parents=True, exist_ok=True)
topologies_dir

In [ ]:
for seed in seeds:
    topologies_seed_dir = os.path.abspath(os.path.join(topologies_dir,f"seed_{seed}"))
    if os.path.exists(topologies_seed_dir):
        print(f"Skipping {topologies_seed_dir=} because it already exists.")
        continue
    topgen_args = TopologyGeneratorArgs(
        model_params_file = pathlib.Path( os.path.join(cloth_root_dir,f"experiments/2024_DLT/PCN_model_params.json") ),
        nb_partitions = [4],
        seed = seed,
        nations = select_eurosystem_subset(["IT","FR","DE","ES"]),
        nb_retail = 400000,
        nb_merchants = 4000,
        nb_intermediaries = 40,
        capacity_fractions = capacities,
        output_dir = pathlib.Path(topologies_seed_dir),
    
        # Other args
        version = False, # Do not print version and exit
        verbose = False,
        p_small_merchants = 0.4,
        p_medium_merchants =  0.3,
        p_large_merchants = 0.3,
        fraction_of_unbanked_retail_users = DEFAULT_FRACTION_OF_UNBANKED_RETAIL_USERS
    )
    topology_generate(topgen_args)


## Run simulations

In [ ]:
results_dir = os.path.abspath(os.path.join(cloth_root_dir,f"experiments/2024_DLT/results"))
results_file = os.path.abspath(os.path.join(results_dir,f"results.csv"))

print(f"{results_dir=}\n{results_file=}")

reset_results = False
if reset_results and os.path.exists(results_dir): 
    shutil.rmtree(results_dir)
pathlib.Path(results_dir).mkdir(parents=True, exist_ok=True)

results = pd.DataFrame()
if os.path.exists(results_file):
    results = pd.read_csv(results_file)

results.head()


### Experiment 1 - Swap Threshold vs Payment Success Rate

In [ ]:
results = run_all_simulations(
    cloth_root_dir = cloth_root_dir,
    topologies_dir = topologies_dir,
    results_dir = results_dir,
    results_file = results_file,
    
    block_congestion_rates = 0.5,
    block_sizes = 4,
    capacities = [0.001, 0.002, 0.005],
    num_processess = 4,
    seeds = seeds, 
    # simulation_ends = 3600000,
    simulation_ends = 86400000,
    submarine_swap_thresholds = [0.6, 0.7, 0.8, 0.9, 0.95, 0.98],
    syncs = 2,
    tpss = 2,
    tps_cfgs = None,
)

### Experiment 2 - Swap Threshold vs Payment Success Rate with shaped load

In [ ]:
results = run_all_simulations(
    cloth_root_dir = cloth_root_dir,
    topologies_dir = topologies_dir,
    results_dir = results_dir,
    results_file = results_file,
    
    block_congestion_rates = 0,
    block_sizes = 4,
    capacities = [0.001, 0.002],
    num_processess = 1,
    seeds = seeds, 
    # simulation_ends = 3600000,
    simulation_ends = 86400000,
    submarine_swap_thresholds = [0.6, 0.8, 0.9],
    syncs = 2,
    tpss = None,
    tps_cfgs = os.path.abspath(os.path.join(experiment_root_dir,f"PCN_load.txt")),
)

### Experiment 3 - Impact of congestion

In [ ]:
results = run_all_simulations(
    cloth_root_dir = cloth_root_dir,
    topologies_dir = topologies_dir,
    results_dir = results_dir,
    results_file = results_file,
    
    block_congestion_rates = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
    block_sizes = 4,
    capacities = [0.001, 0.002, 0.005],
    num_processess = 4,
    seeds = seeds, 
    # simulation_ends = 3600000,
    simulation_ends = 86400000,
    submarine_swap_thresholds = 0.9,
    syncs = 2,
    tpss = 2,
    tps_cfgs = None,
)

## Create charts

In [ ]:
import os
import glob
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from pathlib import Path

# Set up the seaborn style
latex_preamble = r"""
\renewcommand{\bfdefault}{sb}  % Semibold weight
"""
custom_params = {
    'grid.linestyle': '--',
    "text.usetex": True,
    'text.latex.preamble': latex_preamble,
    "font.family": "serif",
    'font.size': 16,
    'legend.title_fontsize': 14,
    'axes.linewidth': 1.5,
    'lines.linewidth': 3,
    'axes.labelsize':  20,
    'xtick.labelsize': 18,
    'ytick.labelsize': 18,
    'legend.fontsize': 16,
    'lines.markersize': 8
}
sns.set_theme(style="whitegrid", rc=custom_params)
sns.color_palette("tab10")

plots_dir = os.path.abspath(os.path.join(cloth_root_dir,f"experiments/2024_DLT/plots"))
pathlib.Path(plots_dir).mkdir(parents=True, exist_ok=True)

### Chart 1

In [ ]:
plot1_df = results[
    (results['tps'] == 2) &
    (results['block_congestion_rate'] == 0)
]
plot1_df.head()

In [ ]:
# Plot 1
# Create a lineplot with twin x-axis
plt.figure(figsize=(10, 5))
ax = sns.lineplot(data=plot1_df,
             x='submarine_swap_threshold',
             y='success',
             marker='o',
             palette='tab10',
             hue="capacity",
             estimator='mean',
             errorbar='sd',
             markersize='12')
ax2 = ax.twinx()
sns.lineplot(data=plot1_df,
                  x='submarine_swap_threshold',
                  y='mean_submarine_swaps_per_minute',
                  hue="capacity",
                  palette='tab10',
                  estimator='mean',
                  errorbar='sd',
                  linestyle='dotted',
                  marker='X',
                  ax=ax2,
                  markersize='8')

# Set labels and title
ax.set_xlabel(r'\textbf{Swap Threshold}',fontsize=22)
ax2.set_ylabel(r'\begin{center}\textbf{Swaps per}\\\textbf{Minute (SPM)}\end{center}',fontsize=22)
ax.set_ylabel(r'\begin{center}\textbf{Payment Success}\\\textbf{Rate (PSR)}\end{center}',fontsize=22)

ax2.xaxis.set_major_locator(ticker.FixedLocator([0.6,0.7,0.8,0.9,0.95,0.98]))
ax.xaxis.set_major_formatter(ticker.PercentFormatter(xmax=1, decimals=0))
ax.yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1, decimals=0))

# Remove legend
ax.legend_.remove()
ax2.legend_.remove()
# Combining legend handles and labels from both axes
handles1, labels1 = ax2.get_legend_handles_labels()
handles2, labels2 = ax.get_legend_handles_labels()
handles = handles2 + handles1
labels = ['800k', '1.6M','4M','800k', '1.6M', '4M']

# Creating the legend
plt.legend(handles, labels, title=r'\begin{center}\textbf{Network Liquidity (\texteuro)}\\\textbf{PSR}\qquad \qquad \textbf{SPM}\end{center}',loc='lower left', fontsize='14',ncol=2)

# Save the plot as a PDF file with tight layout
plt.tight_layout()
plt.savefig(os.path.join(plots_dir, f"plot1_ss-vs-sstp.pdf"),format='pdf',pad_inches = 0,bbox_inches='tight')
plt.show()

## Chart 2 and 3

In [ ]:
plot3_df = results[
    (results['capacity']==0.001) 
    & (results['tps_cfg'] == os.path.abspath(os.path.join(experiment_root_dir,f"PCN_load.txt"))) 
    & (results['block_congestion_rate'] == 0) 
    & ((results['submarine_swap_threshold'] == 0.6) | (results['submarine_swap_threshold'] == 0.8) | (results['submarine_swap_threshold'] == 0.9))
]
plot3_df.head()

In [ ]:
# Step 1: Create goal3 DF from input folder

success_per_minute = pd.DataFrame([
{
    'minute': int(minute), 
    'number': number, 
    'capacity': row['capacity'], 
    'seed': row['seed'], 
    'submarine_swap_threshold': row['submarine_swap_threshold']
}
for _, row in plot3_df.iterrows()
for minute, number in json.loads(row['success_per_minute']).items()
]).sort_values(by=['capacity', 'seed', 'submarine_swap_threshold', 'minute'])

window_size = 15
success_per_minute['success_smooth'] = (success_per_minute.groupby(['capacity','seed','submarine_swap_threshold'])['number'].rolling(window=window_size,min_periods=1).mean().reset_index(level=[0,1,2],drop=True))

tx_per_minute = pd.DataFrame([
    {'minute': int(minute), 'number': number, 'capacity': row['capacity'], 'seed': row['seed'], 'submarine_swap_threshold': row['submarine_swap_threshold']}
    for _, row in plot3_df.iterrows()
    for minute, number in json.loads(row['transactions_per_minute']).items()
]).sort_values(by=['capacity','seed','submarine_swap_threshold', 'minute'])

tx_per_minute['tx_smooth'] = (tx_per_minute.groupby(['capacity', 'seed', 'submarine_swap_threshold'])['number'].rolling(window=window_size,min_periods=1).mean().reset_index(level=[0,1,2],drop=True))

In [ ]:
# Plot 3
fig, (ax1, ax2) = plt.subplots(2, gridspec_kw={'height_ratios': [1, 2]})
fig.set_size_inches(8, 6)

sns.lineplot(x='minute',
        y='success_smooth',
        data=success_per_minute,
        estimator='mean',
        errorbar='sd',
        hue='submarine_swap_threshold',
        ax=ax2,
        palette='tab10')

sns.lineplot(x='minute',
        y='tx_smooth',
        color='gray',
        data=tx_per_minute[tx_per_minute['submarine_swap_threshold']==0.6],
        ax=ax1)


ax2.yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1, decimals=0))
ax2.set_xticks([n for n in range(0,1440,60)])
ax2.set_xticklabels([str(n) for n in range(24)])
ax2.set_xlim(0,1440)
ax2.set_ylim(0.9,1.008)

ax1.set_xticks([n for n in range(0,1440,60)])
ax1.set_xticklabels(['' for n in range(24)])
ax1.set_xlim(0,1440)
ax1.set_yticks([n for n in range(0,450,100)])

ax1.set_xlabel('')
ax2.set_xlabel(r'\textbf{Hour of the Day}')
ax2.set_ylabel(r'\begin{center}\textbf{Payment Success}\\\textbf{Rate}\end{center}',fontsize=22)
ax1.set_ylabel(r'\begin{center}\textbf{Transactions}\\\textbf{per minute}\end{center}',fontsize=22)

handles, labels = ax2.get_legend_handles_labels()
custom_labels = ['60\%', '80\%', '90\%']
ax2.legend(handles, custom_labels, title=r'\textbf{Swap Threshold}', loc='lower left', fontsize='14')

# Save the plot as a PDF file with tight layout
plt.tight_layout()
plt.savefig('./plots/plot3_psr-vs-time.pdf',format='pdf',bbox_inches='tight')

## Chart 4

In [ ]:
goal4_df = results[
    (results['tps']==2) &
    (results['submarine_swap_threshold'] == 0.9)
]
goal4_df.head()

In [ ]:
# Plot 4
plt.figure(figsize=(7, 5))
ax = sns.lineplot(data=goal4_df,
                  x='block_congestion_rate',
                  y='success',
                  hue="capacity",
                  palette='tab10',
                  estimator='mean',
                  errorbar='sd',
                  marker='o',
                  markersize='8')

# Set labels and title
ax.set_xlabel(r'\textbf{Congestion Rate}',fontsize=22)
ax.set_ylabel(r'\textbf{Payment Success Rate}',fontsize=22)

ax.set_xticks([x for x in np.arange(0.1,1,0.1)])
ax.xaxis.set_major_formatter(ticker.PercentFormatter(xmax=1, decimals=0))
ax.yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1, decimals=0))

# Combining legend handles and labels from both axes
handles, labels = ax.get_legend_handles_labels()
custom_labels = ['800k','1.6M','4M']

# Creating the legend
plt.legend(handles, custom_labels, title=r'\textbf{Network Liquidity (\texteuro)}',loc='lower left', fontsize='14')


# Save the plot as a PDF file with tight layout
plt.tight_layout()
plt.savefig('./plots/plot4_psr-vs-cr.pdf',format='pdf',pad_inches = 0,bbox_inches='tight')
plt.show()

## CSV Migrations

In [ ]:
results['sync'] = 2
results.to_csv(results_file, index=False)